In [31]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [32]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

In [33]:
ping('utdt.edu.ar')

Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar


{'dst': 'utdt.edu.ar',
 'response_type': 0,
 'rtt_ms': 89.59221839904785,
 'len': 28,
 'ttl': 56}

In [34]:
def ping_multiple(host, cantidad_paquetes):

    res = dict()
    sent_packets = cantidad_paquetes
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for i in range(cantidad_paquetes):

        resultado = ping(host)

        # print(resultado) # podemos descomentar para ver el resultado de cada ping

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido

    res["host"] = host
    res["Paquetes enviados"] = sent_packets
    res["Paquetes recibidos"] = received_packets
    res["Paquetes perdidos"] = lost_packets
    res["Porcentaje de paquetes perdidos"] = (lost_packets / sent_packets) 
    res["RTT total"] = total_rtt if received_packets > 0 else None
    res["RTT mínimo"] = min_rtt if min_rtt != float('inf') else None
    res["RTT máximo"] = max_rtt if max_rtt != float('-inf') else None
    res["RTT promedio"] = total_rtt / received_packets if received_packets > 0 else None
    res["Desvió estándar del RTT promedio"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos. También contactamos a UTDT para ver las diferencias.

In [35]:
ping_multiple('mit.edu', 5)

Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu


{'host': 'mit.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1330.6891918182373,
 'RTT mínimo': 238.00396919250488,
 'RTT máximo': 286.32116317749023,
 'RTT promedio': 266.13783836364746,
 'Desvió estándar del RTT promedio': 19.66670569347078}

In [36]:
ping_multiple('ie.edu', 5)

Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu


{'host': 'ie.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 238.26193809509277,
 'RTT mínimo': 35.05897521972656,
 'RTT máximo': 58.90989303588867,
 'RTT promedio': 47.652387619018555,
 'Desvió estándar del RTT promedio': 8.625890055580863}

In [37]:
ping_multiple('sydney.edu.au', 5)

Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au


{'host': 'sydney.edu.au',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1900.3939628601074,
 'RTT mínimo': 325.06704330444336,
 'RTT máximo': 538.5589599609375,
 'RTT promedio': 380.0787925720215,
 'Desvió estándar del RTT promedio': 89.67579550472396}

In [38]:
ping_multiple('cuhk.edu.cn', 5)

Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
No se recibió respuesta de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn


{'host': 'cuhk.edu.cn',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 4,
 'Paquetes perdidos': 1,
 'Porcentaje de paquetes perdidos': 0.2,
 'RTT total': 1431.286334991455,
 'RTT mínimo': 322.9548931121826,
 'RTT máximo': 435.84418296813965,
 'RTT promedio': 357.82158374786377,
 'Desvió estándar del RTT promedio': 52.4545285283615}

In [39]:
ping_multiple('uniben.edu', 5)

Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu


{'host': 'uniben.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 148.4239101409912,
 'RTT mínimo': 23.12493324279785,
 'RTT máximo': 42.3431396484375,
 'RTT promedio': 29.684782028198242,
 'Desvió estándar del RTT promedio': 7.50662480645388}